# Check incidence and elevation angle calculations

This notebooks calculates the incidence and elevation angles at a specific grid point in the RUNW geolocation grid using the corresponding slant range, zero Doppler time, elevation, and projected x, y coordinates.  

These numbers should agree perfectly (at least to several significant digits). Instead there are differences of a few hundreths of a degree.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import nisarHDF
import matplotlib.pyplot as plt
import numpy as np
import geojson
import os
from scipy import optimize
import pyproj
from datetime import datetime
import h5py
import scipy

In [2]:
hdfFile = 'https://nisar.asf.alaska.edu/NISAR-SAMPLE-DATA/RUNW/ALOS-1_Rosamond_20081012_20081127/NISAR_L1_PR_RUNW_001_005_A_219_220_4020_SH_20081012T060910_20081012T060926_20081127T060959_20081127T061015_P01101_M_F_J_001.h5'
if not os.path.exists(os.path.basename(hdfFile)):
    !wget {hdfFile} 

In [3]:
#hdfFile = '../hdfTest/icesheet/data_pair2_ALOS2497735530-230811_ALOS2499805530-230825_14d/RUNW_0000526644_001001_ALOS2497735530-230811_0000528214_001001_ALOS2499805530-230825.h5'
hdfFile = 'NISAR_L1_PR_RUNW_001_005_A_219_220_4020_SH_20081012T060910_20081012T060926_20081127T060959_20081127T061015_P01101_M_F_J_001.h5'
h5 = h5py.File(hdfFile, 'r')
geolocation = h5['science']['LSAR']['RUNW']['metadata']['geolocationGrid']

## Get Statevectors

Function to parse the state vectors and setup interpolators.

In [4]:
def parseStateVectors(h5):
    orbit = h5['science']['LSAR']['RUNW']['metadata']['orbit']
    sv = {}
    time = np.array(orbit['time'])
    position = np.array(orbit['position'])
    velocity = np.array(orbit['velocity'])
    for i, pos, vel in zip(range(0, 3), ['xsv', 'ysv', 'zsv'], ['vxsv', 'vysv', 'vzsv']):
        sv[pos]= scipy.interpolate.RegularGridInterpolator(
                    [time], position[:, i], method='quintic')
        sv[vel]= scipy.interpolate.RegularGridInterpolator(
                    [time], velocity[:, i], method='quintic')
    return sv

Parse the state vectors.

In [5]:
sv = parseStateVectors(h5)

## Get Grid

Get the gelocation coordinates and incidence and elevationAngles.

In [6]:
epsg = np.array(geolocation['epsg']).item()
print(f'EPSG = {epsg}')
slantRange = np.array(geolocation['slantRange'])
dR = slantRange[1]- slantRange[0]
print(f'Slant Range Spacing {dR:.2f}')
xCoord = np.array(geolocation['coordinateX'])
yCoord = np.array(geolocation['coordinateY'])
zeroDopplerTime = np.array(geolocation['zeroDopplerTime'])
heightAboveEllipsoid = np.array(geolocation['heightAboveEllipsoid'])
#
elevationAngle =  np.array(geolocation['elevationAngle'])
incidenceAngle = np.array(geolocation['incidenceAngle'])
#
print(elevationAngle.shape, heightAboveEllipsoid.shape, zeroDopplerTime.shape, slantRange.shape)

EPSG = 32611
Slant Range Spacing 499.65
(21, 251, 58) (21,) (251,) (58,)


Get values for a specific point (0 elevation)

In [7]:
iz, it, ir = 2, 0, 0
zPt = heightAboveEllipsoid[iz]
# range/Doppler
slantRangePt = slantRange[ir]
zeroDopplerPt = zeroDopplerTime[it]
# angles
elevationAnglePt = elevationAngle[iz, it, ir]
incidenceAnglePt = incidenceAngle[iz, it, ir]
# projected coordinates
xPt, yPt = xCoord[iz, it, ir], yCoord[iz, it, ir]
#
print(f'x, y, z = {xPt:.1f} {yPt:.1f} {zPt:.1f} (m), zeroDopplerTime=  {zeroDopplerPt:.4f} (s), '
      f'slantRange = {slantRangePt:.2f} (m), elevationAngle {elevationAnglePt:.4f} incidenceAngle {incidenceAnglePt:.4f}')

x, y, z = 385143.5 3789475.6 0.0 (m), zeroDopplerTime=  22150.3320 (s), slantRange = 743587.39 (m), elevationAngle 18.9604 incidenceAngle 21.1295


Convert target to ECEF and get the corresponding radius.

In [8]:
# Setup conversion from projected coordinates to ECEF 
XYToECEF = pyproj.Transformer.from_crs(f"EPSG:{epsg}", "EPSG:4978").transform
ptECEF = XYToECEF(xPt, yPt, zPt)
ptRadius = np.linalg.norm(ptECEF)
print(f'Radius to pt target at elevation {zPt} m =  {ptRadius:.2f} m')

Radius to pt target at elevation 0 m =  6371405.66 m


Get the satellite position for the zeroDoppler time and compute the corresponding radius.

In [9]:
satPosition = [sv[coord]([zeroDopplerPt])[0] for coord in ['xsv', 'ysv', 'zsv']]
print(satPosition)
ReH = np.linalg.norm(satPosition)
print(f'Radius from Earth Center to Sat {ReH:.2f}')

[-3015234.5030060257, -5058048.299101401, 3913152.8595616613]
Radius from Earth Center to Sat 7070237.41


Paul's solution

In [10]:
r_b_t = ptRadius
r_b_sc_plus_h = ReH 
rho = slantRangePt
bta =  np.arccos((0.5*r_b_t/(r_b_sc_plus_h))+0.5*((r_b_sc_plus_h)/r_b_t)-(0.5*(rho/(r_b_sc_plus_h))*(rho/r_b_t)))
#  cthta = (r_b_t**2 - r_b_sc_plus_h**2 - rho**2)/ (-2. *  r_b_t * rho)
cthta = (rho**2 + r_b_sc_plus_h**2 - r_b_t**2 ) / (2. *  r_b_sc_plus_h * rho)
thta = np.arccos(cthta)
print(f"beta  = {bta*180./np.pi:.4f}")
print(f"theta = {thta*180./np.pi:.4f}")
print(f"incidence angle = {(thta+bta)*180./np.pi:.4f}")

beta  = 2.1691
theta = 18.9238
incidence angle = 21.0929


GrIMP solution.

In [11]:
elevationAngleCalc = np.arccos((slantRangePt**2 + ReH**2 - ptRadius**2) / (2.0 * slantRangePt * ReH))
incidenceAngleCalc = np.arcsin(np.sin(elevationAngleCalc) * ReH/ptRadius)

# Summary

Extracting x, y, z, slantRange, and zeroDoppler from the geolocation group and using those values to calculate the incidence and elevation angles produces a result that differs from the corresponding values in the gelociation grid. 

In this particular example, the error corresponds to either and elevation error of +171 m or a slant range error of +183.5 m. With one of the ALOS Antarctica scenes the error is bit large (0.04742) and corrections of ~400 m are needed to force agreement.

Given that the slant range spacing is not an integer multiple of 500 (499.22 in this case), I suspect it has something to do with the way the slant range coordinates are determined (see seperate issue regarding errors in the zero Doppler time grid).

In [14]:
print('Calculated 1')
print(f'elevationAngle = {np.degrees(elevationAngleCalc):.4f}, incidenceAngle={np.degrees(incidenceAngleCalc):.4f}')
print('Calculated 2')
print(f'elevationAngle = {thta*180./np.pi:.4f}, incidenceAngle={(thta+bta)*180./np.pi:.4f}')
print('From Table')
print(f'elevationAngle = {elevationAnglePt:.4f}, incidenceAngle={incidenceAnglePt:.4f}')

Calculated 1
elevationAngle = 18.9238, incidenceAngle=21.0929
Calculated 2
elevationAngle = 18.9238, incidenceAngle=21.0929
From Table
elevationAngle = 18.9604, incidenceAngle=21.1295


In [15]:
print(f'Difference with calculated 1 {np.degrees(elevationAngleCalc) - elevationAnglePt:.5f} {np.degrees(incidenceAngleCalc) - incidenceAnglePt:.5f}')
print(f'Difference with calculated 2 {np.degrees(thta) - elevationAnglePt:.5f} {np.degrees(thta + bta) - incidenceAnglePt:.5f}')

Difference with calculated 1 -0.03660 -0.03657
Difference with calculated 2 -0.03660 -0.03657
